In [8]:
import numpy as np

In [ ]:
def sigmoid(x):
    """
    Sigmoid 函数的实现
    
    参数:
    x: 输入值，可以是标量、向量或矩阵
    
    返回:
    result: 对应于 x 的 Sigmoid 函数值
    """
    result = 1 / (1 + np.exp(-x))
    return result

In [ ]:
def relu(x):
    """
    ReLU 激活函数的实现
    
    参数:
    x: 输入值，可以是标量、向量或矩阵
    
    返回:
    result: 对应于 x 的 ReLU 激活函数值
    """
    result = np.maximum(0, x)
    return result

In [ ]:
def tanh(x):
    """
    Tanh 激活函数的实现
    
    参数:
    x: 输入值，可以是标量、向量或矩阵
    
    返回:
    result: 对应于 x 的 Tanh 激活函数值
    """
    result = (np.exp(x) - np.exp(-x)) / (np.exp(x) + np.exp(-x))
    return result

In [102]:
def softmax(z):
    """
    Softmax 函数的实现
    
    参数:
    z: 输入向量，可以是任意长度的一维数组，或二维数组
    
    返回:
    result: 对应于 z 的 Softmax 函数值
    """
    # 为了数值稳定性，减去最大值
    if len(z.shape) == 1:
        exp_z = np.exp(z - np.max(z, axis=-1))  
        result = exp_z / np.sum(exp_z, axis=-1)
        return result
    if len(z.shape) == 2:
        r, c = z.shape
        exp_z = np.exp(z - np.max(z, axis=-1).reshape(r, 1))
        result = exp_z / np.sum(exp_z, axis=-1).reshape(r, 1)
        return result

In [113]:
def relu(x):
    x = np.maximum(0, x)
    return x

In [122]:
class BatchNormalization:
    def __init__(self, epsilon=1e-5, momentum=0.9):
        self.epsilon = epsilon
        self.momentum = momentum
        self.running_mean = None
        self.running_var = None
        self.gamma = None
        self.beta = None

    def forward(self, x, training=True):
        if self.running_mean is None:
            N, D = x.shape
            self.running_mean = np.zeros(D)
            self.running_var = np.zeros(D)
            self.gamma = np.ones(D)
            self.beta = np.zeros(D)

        if training:
            sample_mean = np.mean(x, axis=0)
            sample_var = np.var(x, axis=0)
            x_normalized = (x - sample_mean) / np.sqrt(sample_var + self.epsilon)

            self.running_mean = self.momentum * self.running_mean + (1 - self.momentum) * sample_mean
            self.running_var = self.momentum * self.running_var + (1 - self.momentum) * sample_var

            self.x_normalized = x_normalized
        else:
            x_normalized = (x - self.running_mean) / np.sqrt(self.running_var + self.epsilon)

        out = self.gamma * x_normalized + self.beta
        return out

    def backward(self, dout):
        N = dout.shape[0]

        dx_normalized = dout * self.gamma
        dvar = np.sum(dx_normalized * (self.x_normalized - self.running_mean) * -0.5 * (self.running_var + self.epsilon)**(-1.5), axis=0)
        dmean = np.sum(dx_normalized * -1 / np.sqrt(self.running_var + self.epsilon), axis=0) + dvar * np.sum(-2 * (self.x_normalized - self.running_mean), axis=0) / N
        dx = dx_normalized / np.sqrt(self.running_var + self.epsilon) + dvar * 2 * (self.x_normalized - self.running_mean) / N + dmean / N

        self.dgamma = np.sum(dout * self.x_normalized, axis=0)
        self.dbeta = np.sum(dout, axis=0)

        return dx


In [131]:
def convolution2d(image, kernel):
    """
    二维卷积操作
    
    参数:
    image: 输入的二维图像，形状为 (H, W)，其中 H 为图像高度，W 为图像宽度
    kernel: 卷积核，形状为 (K, K)，其中 K 为卷积核的大小，通常为奇数
    
    返回:
    result: 卷积后的图像，形状为 (H-K+1, W-K+1)
    """
    # 获取图像和卷积核的形状信息
    image_height, image_width = image.shape
    kernel_size = kernel.shape[0]
    
    # 计算卷积后的图像大小
    result_height = image_height - kernel_size + 1
    result_width = image_width - kernel_size + 1
    
    # 初始化卷积后的图像
    result = np.zeros((result_height, result_width))
    
    # 进行卷积操作
    for i in range(result_height):
        for j in range(result_width):
            result[i, j] = np.sum(image[i:i+kernel_size, j:j+kernel_size] * kernel)
    
    return result

In [138]:
def binary_cross_entropy(y, y_pred):
    """
    二分类交叉熵损失函数的实现
    
    参数:
    y: 真实的标签向量，形状为 (N,)
    y_pred: 模型的预测概率向量，形状为 (N,)
    
    返回:
    loss: 二分类交叉熵损失值
    """
    epsilon = 1e-12  # 避免log(0)导致的数值不稳定
    loss = -np.mean(y * np.log(y_pred + epsilon) + (1 - y) * np.log(1 - y_pred + epsilon))
    return loss

def cross_entropy(y, y_pred):
    """
    多分类交叉熵损失函数的实现
    
    参数:
    y: 真实的标签向量，形状为 (N, C)，其中 N 是样本数量，C 是类别数量
    y_pred: 模型的预测概率矩阵，形状为 (N, C)
    
    返回:
    loss: 多分类交叉熵损失值，形状为 (N,)
    """
    epsilon = 1e-12  # 避免log(0)导致的数值不稳定
    loss = -np.mean(y * np.log(y_pred + epsilon))
    return loss

In [151]:
def focal_loss(y_true, y_pred, alpha=0.25, gamma=2):
    """
    Focal Loss 的多分类版本实现
    
    参数:
    y_true: 真实的标签矩阵，形状为 (N, C)，其中 N 是样本数量，C 是类别数量，采用 one-hot 编码
    y_pred: 模型的预测概率矩阵，形状为 (N, C)，表示每个样本属于各个类别的概率
    alpha: 正样本和负样本的权重系数，默认为 0.25
    gamma: 调节参数，默认为 2
    
    返回:
    loss: Focal Loss 的值
    """
    epsilon = 1e-12
    p_t = np.clip(y_pred, epsilon, 1.0 - epsilon) # 避免 log(0) 导致的数值不稳定
    loss = -alpha * (1 - p_t) ** gamma * y_true * np.log(p_t)
    
    return loss.mean()

In [152]:
# 示例用法
y_true = np.array([1, 0, 1, 0, 1]) # 真实的标签向量
y_pred = np.array([0.8, 0.2, 0.6, 0.3, 0.9]) # 模型的预测概率向量

# 计算 Focal Loss
loss = focal_loss(y_true, y_pred)
print("Focal Loss:", loss)

Focal Loss: 0.004585572350585259


2.0

In [156]:
1e-4

0.0001